In [ ]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'15',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '4613d2f4-2b10-46be-a5e3-8226d6ac3757',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

#NOTE:
# I had to go in and put "jupyter notebook --NotebookApp.iopub_data_rate_limite=1e10"
# Into Anaconda Prompt to change this to allow to pull data
# Then had to use the local host URL provided by Anaconda Prompt

In [ ]:
type(data)

In [ ]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)

In [ ]:
#This normalises the data and makes it more readable

df = pd.json_normalize(data['data'])
df['timestamp'] = pd.to_datetime('now')
df

In [ ]:
def api_runner():
    global df
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest' 
    #Original Sandbox Environment: 'https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
      'start':'1',
      'limit':'15',
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': '4613d2f4-2b10-46be-a5e3-8226d6ac3757',
    }

    session = Session()
    session.headers.update(headers)

    try:
      response = session.get(url, params=parameters)
      data = json.loads(response.text)
      #print(data)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
      print(e)
            
#NOTE:
# I had to go in and put "jupyter notebook --NotebookApp.iopub_data_rate_limite=1e10"
# Into Anaconda Prompt to change this to allow to pull data
# Then had to use the local host URL provided by Anaconda Prompt

# Use this if you just want to keep it in a dataframe
    # df2 = pd.json_normalize(data['data'])
    # df2['timestamp'] = pd.to_datetime('now')
    # df = pd.concat([df, df2], ignore_index=True)

# Use this to have data in csv format
    df = pd.json_normalize(data['data'])
    df['timestamp'] = pd.to_datetime('now')
    df

    if not os.path.isfile(r'C:\Users\kholb\OneDrive\Documents\Python Scripts\API.csv'):
        df.to_csv(r'C:\Users\kholb\OneDrive\Documents\Python Scripts\API.csv', header='column_names')
    else:
        df.to_csv(r'C:\Users\kholb\OneDrive\Documents\Python Scripts\API.csv', mode='a', header=False)


In [ ]:
import os
from time import time
from time import sleep

for i in range(333):
    try:
        api_runner()
        print('API Runner completed successfully')
        sleep(60) #sleep for 1 minute
    except Exception as e:
        print(f"Error in main loop: {e}")
        continue
exit()

In [ ]:
df3 = pd.read_csv(r'C:\Users\kholb\OneDrive\Documents\Python Scripts\API.csv')
df3

In [ ]:
df

In [ ]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [ ]:
df

In [ ]:
df4 = df.groupby('name', sort=False)[['quote.USD.percent_change_1h','quote.USD.percent_change_24h','quote.USD.percent_change_7d', 'quote.USD.percent_change_30d', 'quote.USD.percent_change_60d','quote.USD.percent_change_90d']].mean()
df4

In [ ]:
df5 = df4.stack()
df5

In [ ]:
type(df5)

In [ ]:
df6 = df5.to_frame(name='values')
df6

In [ ]:
df6.count()

In [ ]:
index = pd.Index(range(90))

df7 = df6.reset_index()
df7

In [ ]:
df8 = df7.rename(columns={'level_1': 'percent_change'})
df8

In [ ]:
df8['percent_change'] = df8['percent_change'].replace(['quote.USD.percent_change_1h', 'quote.USD.percent_change_24h','quote.USD.percent_change_7d','quote.USD.percent_change_30d','quote.USD.percent_change_60d','quote.USD.percent_change_90d'],['1h','24h','7d','30d','60d','90d'])
df8

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.catplot(x='percent_change', y='values', hue='name', data=df8, kind='point')

In [ ]:
df9 = df[['name','quote.USD.price','timestamp']]
df9 = df9.query("name == 'Bitcoin'")
df9